# Inference for Citation Extraction

This notebook is for sanity-checking and testing inference for the citation and extraction model.

In [1]:
%load_ext autoreload

## Imports

Import whatever we need for running inference. For now, this is just the function
to output raw labels from a string input. It's also worth checking what the default
loading location is for the model.

In [2]:
from transformers import AutoModelForTokenClassification

from perscit_model.extraction.inference import MODEL_SAVE_DIR, InferenceModel

print(MODEL_SAVE_DIR)

/home/archytas/Projects/perseus-citation-model/outputs/models/extraction


## Expected input text without `<bibl>`, `<cit>`, and `<quote>` tags

The cell below tests inference on text input in the expected format, with all XML tags that explicitly give
the desired labels removed. We can see that it works quite well. This text is from the training data itself,
so we should also look at text from the test data.

TODO: reconstruct XML tags based on labels and reconstruct string. Give metrics.

In [3]:
text = "direct or reported, is a regular Greek idiom. Cf. Plato, Rep. 332 D<foreign lang=\"greek\">h( ti/si ti/ a)podidou=sa te/xnh</foreign>; Soph. <title>El.</title> 751 oi(=' e)/rga dra/sas oi(=a lagxa/nei kaka/. In instances like the present"
model = InferenceModel()
tokens, labels = model.process_text(text)
print(labels)

/home/archytas/Projects/perseus-citation-model/.venv/lib/python3.13/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'I-BIBL', 'B-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'I-QUOTE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [4]:
model.insert_tags_into_xml(text, tokens, labels)

'direct or reported, is a regular Greek idiom. Cf. Plato, Rep. 332 D<foreign lang="greek">h( ti/si ti/ a)podidou=sa te/xnh</foreign>;<bibl> Soph. <title>El.</title> 751</bibl><quote> oi(=\' e)/rga dra/sas oi(=a lagxa/nei kaka/.</quote> In instances like the present'

## Text with `<bibl>`, `<cit>`, and `<quote>` tags

This tests the behaviour of model inference where the labels are present in the data
as XML tags. This is mostly just to diagnose data leakage issues: it's not in itself very important
whether the model identifies these tokens correctly or not.

What we see here is that the model overlooks anything between`<bib>`, `<cit>`, and `<quote>` tags.
This does suggest that the model is looking for a pretty precise format for identifying these elements:
including explicit labels as XML tags is enough to throw off inference. Because the model ignores
citations where they are already labelled as such in the XML, it might be possible to process
XML documents simply by passing them through the model and inserting tags accordingly. This will work
as long as the model consistently ignores already-tagged citations.

In [5]:
text = "direct or reported, is a regular Greek idiom. Cf. <bibl n=\"Plat. Rep. 332D\">Plato, Rep. 332 D</bibl> <foreign lang=\"greek\">h( ti/si ti/ a)podidou=sa te/xnh</foreign>;  <cit><bibl n=\"Soph. El. 751\">Soph. <title>El.</title> 751</bibl> <quote lang=\"greek\">oi(=' e)/rga dra/sas oi(=a lagxa/nei kaka/</quote></cit>. In instances like the present"
tokens, labels = model.process_text(text)
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [7]:
model.insert_tags_into_xml(text, tokens, labels)

'direct or reported, is a regular Greek idiom. Cf. <bibl n="Plat. Rep. 332D">Plato, Rep. 332 D</bibl> <foreign lang="greek">h( ti/si ti/ a)podidou=sa te/xnh</foreign>;  <cit><bibl n="Soph. El. 751">Soph. <title>El.</title> 751</bibl> <quote lang="greek">oi(=\' e)/rga dra/sas oi(=a lagxa/nei kaka/</quote></cit>. In instances like the present'

In [ ]:
tokens["input_ids"].dim()

In [ ]:
token_ids = [ids[mask] for ids, mask in zip(tokens["input_ids"], tokens["attention_mask"].bool())]

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("microsoft/deberta-v3-base")

In [ ]:
model.config